# MNIST Convolutional Neural Network in Keras

Bulding a Convolutional Neural Network to classify MNIST digits.

#### Load dependencies and make reproducible

In [1]:
import numpy as np
np.random.seed(42)

import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, LeakyReLU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Load MNIST data

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Basic Preprocessing
Normalise it

In [3]:
X_train = X_train.reshape(-1, 28, 28, 1).astype("float32")/255.
X_test = X_test.reshape(-1, 28, 28, 1).astype("float32")/255.

#### One-hot encode the output

In [4]:
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### Neural Network architecture

`Conv2D` is a 2D convolution layer. On this case, a spatial convolution over images.
This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. `padding = same` results in padding the input so that the output has the same length as the original input.

`MaxPooling2D` does the max pooling operation for spatial data. Why do we _pool_ it? It is common to periodically insert a Pooling layer in-between successive layers in a CNN. Its function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network, and hence to also control overfitting. The Pooling Layer operates independently on every depth slice of the input and resizes it spatially, using the MAX operation. The most common form is a pooling layer with filters of size 2x2 applied with a stride of 2 downsamples every depth slice in the input by 2 along both width and height, discarding 75% of the activations. Every MAX operation would in this case be taking a max over 4 numbers (little 2x2 region in some depth slice). The depth dimension remains unchanged.
The first argument passed to the Conv2D() layer function is the number of output channels and the next input is the kernel_size (a single integer to specify the same value for all spatial dimensions or tuple if not the case).



In [5]:
model = Sequential()

model.add(Conv2D(32, 7, padding = 'same', input_shape = (28, 28, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, 3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.30))

model.add(Conv2D(128, 2, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.50))

model.add(Dense(10, activation = "softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        1600      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         32896     
__________

#### Callbacks

In [6]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-cnn-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')

tensorboard = TensorBoard("logs/03_convnet-mnist")

#### Configure model

In [7]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [8]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 1, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 70s 1ms/step - loss: 0.3523 - acc: 0.8855 - val_loss: 0.0575 - val_acc: 0.9833


#### Test Predictions

In [9]:
saved_model = keras.models.load_model('model_output/weights-cnn-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)

#### Test Final Accuracy

In [10]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 4s 434us/step
Final loss: 0.0574, final accuracy: 0.9823
